In [ ]:
from dotenv import load_dotenv
load_dotenv()

import logging
import sys
date_strftime_format = "%Y-%m-%y %H:%M:%S"
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(message)s", datefmt=date_strftime_format)

from falkordb_gemini_kg.classes.model_config import KnowledgeGraphModelConfig
from falkordb_gemini_kg import KnowledgeGraph, Ontology
from falkordb_gemini_kg.classes.source import Source
from falkordb_gemini_kg.models.openai import OpenAiGenerativeModel
import vertexai
import os
from random import shuffle
import json
from falkordb import FalkorDB

# Initialize the Vertex AI client
vertexai.init(project=os.getenv("PROJECT_ID"), location=os.getenv("REGION"))

### Import source data

In [ ]:
src_files = "data/fight"
sources = []

# For each file in the source directory, create a new Source object
for file in os.listdir(src_files):
    sources.append(Source(os.path.join(src_files, file)))

### Automatically create the ontology from the sources

In [ ]:
boundaries = """
    Extract only the most information about the fighters, fights, and events in the UFC.
    Do not create entities for what can be expressed as attributes.
"""

model = OpenAiGenerativeModel(model_name="gpt-4o")

ontology = Ontology.from_sources(
    sources=sources[: round(len(sources) * 0.1)],
    boundaries=boundaries,
    model=model,
)


db = FalkorDB()
graph = db.select_graph("ufc_ontology")
ontology.save_to_graph(graph)

# Save ontology to json file
with open("ontologies/ufc_ontology.json", "w", encoding="utf-8") as file:
    file.write(json.dumps(ontology.to_json(), indent=2))

### Read ontology from json file

In [ ]:
ontology_file = "ontologies/ufc_ontology_corrected.json"
with open(ontology_file, "r", encoding="utf-8") as file:
    ontology = Ontology.from_json(json.loads(file.read()))

db = FalkorDB()
graph = db.select_graph("ufc_ontology")
ontology.save_to_graph(graph)

kg = KnowledgeGraph(
    name="ufc",
    model_config=KnowledgeGraphModelConfig.with_model(model),
    ontology=ontology,
)

### Process the sources raw data into the knowledge graph

In [ ]:
kg.process_sources(sources)

### Ask a single question to the model

In [ ]:
kg.ask("What were the last 5 fights? When were they? How many rounds did they have?")

In [ ]:
kg.ask("How many takedowns did Alexandre Pantoja have in all fights?")

### Start a chat session with the model

In [ ]:
chat = kg.chat_session()

print(chat.send_message("Who is Salsa Boy?"))